In [ ]:
#   Copyright (c) 2020, Xilinx, Inc.
#   All rights reserved.
#
#   Redistribution and use in source and binary forms, with or without
#   modification, are permitted provided that the following conditions are met:
#
#   1.  Redistributions of source code must retain the above copyright notice,
#       this list of conditions and the following disclaimer.
#
#   2.  Redistributions in binary form must reproduce the above copyright
#       notice, this list of conditions and the following disclaimer in the
#       documentation and/or other materials provided with the distribution.
#
#   3.  Neither the name of the copyright holder nor the names of its
#       contributors may be used to endorse or promote products derived from
#       this software without specific prior written permission.
#
#   THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
#   AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO,
#   THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR
#   PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR
#   CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL,
#   EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO,
#   PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS;
#   OR BUSINESS INTERRUPTION). HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY,
#   WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR
#   OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF
#   ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

!/usr/bin/env python
# coding: utf-8

# SysMon Example Script 
# =====================

# Description
# -----------
# This demo shows how to do four things:
# 1. Connect to a Versal target device via ChipScope Server (cs_server)
#    and Hardware Server (hw_server)
# 2. Program a Versal target device using a design PDI file
# 3. Discover Design Specifics and setup measurements
# 4. Plot live data
#
# Requirements
# ------------
# The following is required to run this demo:
# 1. Local or remote access to a Versal device
# 2. 2021.1+ cs_server and hw_server applications
# 3. Python 3.8 environment
# 4. chipscopy package installed in your environment
#
# ---

# ## Step 1 - Set up environment

In [ ]:


import os

# Specify locations of the running hw_server and cs_server below.
from time import sleep
from more_itertools import one


CS_URL = os.getenv("CS_SERVER_URL", "TCP:localhost:3042")
HW_URL = os.getenv("HW_SERVER_URL", "TCP:localhost:3121")
BOARD = os.getenv("HW_SERVER_BOARD", "vck190/production/2.0")

EXAMPLES_DIR = os.path.realpath(os.getcwd() + "/..")
LTX_FILE = f"{EXAMPLES_DIR}/designs/{BOARD}/ks_demo/ks_demo_wrapper.ltx"
PDI_FILE = f"{EXAMPLES_DIR}/designs/{BOARD}/ks_demo/ks_demo_wrapper.pdi"
print(f"HW_URL={HW_URL}")
print(f"CS_URL={CS_URL}")


# ## Step 2 - Create a session and connect to the server(s)
# Here we create a new session and print out some versioning information for diagnostic purposes.

In [ ]:


from chipscopy import __version__, dm
from chipscopy import create_session, report_versions

print(f"Using chipscopy api version: {__version__}")
session = create_session(cs_server_url=CS_URL, hw_server_url=HW_URL)
report_versions(session)
print()


# ## Step 3 - Get our device from the session
# The session is a container that keeps track of devices and debug cores.

In [ ]:

# Use the first available device and setup its debug cores
if len(session.devices) == 0:
    print("\nNo devices detected")
    exit()

print(f"Device count: {len(session.devices)}")
versal_device = session.devices[0]


# ## Discover Cores and Get Running Design Info
#
# 1. We begin by enumerating the debug cores (hard and soft) present in the design.
# 1. Then we ask the design for the supported timebases. And, finally:
# 1. The NoC is scanned to determine the activated elements.

In [ ]:


# set this to false if you do not want to program the device
program_device = True
program_device = False
if program_device:
    print(f"Programming {PDI_FILE}...")
    versal_device.program(PDI_FILE)

print(f"Discovering debug cores...", end="")
versal_device.discover_and_setup_cores(sysmon_scan=True)
print("Complete!")

sysmon = one(versal_device.sysmon_root)


all_sensors = sysmon.get_all_sensors()
for sensor in all_sensors:
    print(f"{sensor}, ", end="")
print()
print()
print("initializing sensors")
active_nodes = sysmon.initialize_sensors()
print("refresh measurement schedule")
schedule = sysmon.refresh_measurement_schedule()
print("Sensors:")
for sensor in schedule.values():
    print(f"  {sensor}")


class SysMonNodeListener(dm.NodeListener):
    def node_changed(self, node, updated_keys):
        if "device_temp" in node.props:
            print(f"Device Temp: {node.props['device_temp']}")
        for supply_idx, named_sensor in schedule.items():
            supply = f"supply{supply_idx}"
            if supply in node.props:
                print(f"{named_sensor}: {node.props[supply]}")
        print()


node_listener = SysMonNodeListener()
session.chipscope_view.add_node_listener(node_listener)

sysmon.stream_sensor_data(1000)

while True:
    session.chipscope_view.run_events()
    sleep(0.1)

print("done")